In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import numpy as np

In [2]:
url = 'https://rl.fx678.com/date/20190413.html'
headers = {'User-Agent': 
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

r = requests.get(url, headers = headers)
html = r.text.encode(r.encoding).decode()
soup = BeautifulSoup(html, 'lxml')

In [3]:
# 查找所需要对象
table = soup.find('table', id = 'current_data')

In [4]:
# 查看行号
height = len(table.findAll(lambda tag:tag.name == 'tr' and
                          len(tag.findAll('td')) >= 1))
print(height)

# 查看列数
for row in table.findAll('tr'):
    print(len(row.findAll('td')), end='\t')

7
0	0	9	7	7	7	7	7	7	

In [5]:
# 收集表头
columns = [x.text for x in table.tr.findAll('th')]
columns = [x.replace('\xa0', ' ') for x in columns]
columns

['时间', '区域', '指标', '前值', '预测值', '公布值', '重要性', '利多 利空', '解读']

In [6]:
# 构造dataFrame，准备存储表格
width = len(columns)
df = pd.DataFrame(data = np.full((height, width), ' ', dtype = 'U'), columns = columns)
rows = [row for row in table.findAll('tr') if row.find('td') != None]


In [12]:
# 逐行解析表格
for i in range(len(rows)):
    cells = rows[i].findAll('td')
    
    # 如果该行单元格数量于dataFrame列数相同
    if len(cells) == width:
        df.iloc[i] = [cell.text.replace(' ','').replace('\n', '') for cell in cells]
        
        # 若单元格跨多行，则进行多行填充
        for j in range(len(cells)):
            if cells[j].has_attr('rowspan'):
                z = int(cells[j].attrs['rowspan'])
                df.iloc[i:i+z, j] = [cells[j].text.replace(' ','').replace('\n', '')] * z
    else:
        w = len(cells)
        df.iloc[i, width-w:] = [cell.text.replace(' ','').replace('\n', '') for cell in cells]
df.to_excel("20190414财经日历.xlsx", encoding='utf8', index=False)